In [ ]:
!pip install torch
!pip install transformers datasets
!pip install peft # The PEFT library from Hugging Face
!pip install bitsandbytes # For loading model in 4-bit
!pip install accelerate

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from accelerate import Accelerator

# Use Accelerate to handle device placement
accelerator = Accelerator()

# Load the base model and tokenizer
model_name = "Salesforce/codegen-350M-mono"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(model_name)

# Move model to the correct device using Accelerate
base_model = accelerator.prepare(base_model)

Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

In [ ]:
prompt = "Write a Python function to implement bubble sort."

# The tokenizer needs a device, get it from accelerator
inputs = tokenizer(prompt, return_tensors="pt").to(accelerator.device)
#print(inputs)

print("--- BASE MODEL OUTPUT (BEFORE FINE-TUNING) ---")

# Generate text
outputs = base_model.generate(
    **inputs,
    max_new_tokens=80,
    pad_token_id=tokenizer.eos_token_id # Set pad token for open-ended generation
)

before_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(before_output)



--- BASE MODEL OUTPUT (BEFORE FINE-TUNING) ---
Write a Python function to implement bubble sort.

# The function should return a sorted list.
# The list should be passed as an argument to the function.

# For example,
# bubble_sort([5, 4, 2, 1, 3]) should return [1, 2, 3, 4, 5].

# For this function, you may assume that each element in the list has a unique



In [ ]:
# You will need these libraries:
# pip install peft datasets

from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import torch

# 1. Load and Prepare Dataset
dataset = load_dataset("sahil2801/CodeAlpaca-20k", split="train[:10000]") # Load the entire training split

def format_and_tokenize(example):
    # Format prompt and tokenize
    text = f"Instruction: {example['instruction']}\nOutput: {example['output']}"
    return tokenizer(text, truncation=True, max_length=128) # Keep max_length reasonable

tokenized_dataset = dataset.map(format_and_tokenize, remove_columns=dataset.column_names)

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

# 2. Apply LoRA Configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["qkv_proj"], # Target attention layers in CodeGen
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
peft_model = get_peft_model(base_model, lora_config)
peft_model.print_trainable_parameters()


# 3. Define Trainer and Train
training_args = TrainingArguments(
    output_dir="codegen-python-finetuned",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    logging_steps=100,
    learning_rate=3e-4,
    fp16=torch.cuda.is_available(), # Use mixed precision if on GPU
    report_to="none", # Disable wandb logging
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset,
    # Data collator handles padding for language modeling
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

print("\n--- STARTING LoRA FINE-TUNING ---")
trainer.train()

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


trainable params: 1,310,720 || all params: 358,023,168 || trainable%: 0.3661

--- STARTING LoRA FINE-TUNING ---


Step,Training Loss
100,1.202300
200,1.095700
300,1.053300
400,1.052700
500,1.052000
600,1.069900
700,1.034500
800,1.021700
900,1.003900
1000,1.032800


TrainOutput(global_step=1250, training_loss=1.0499560028076171, metrics={'train_runtime': 249.7793, 'train_samples_per_second': 40.035, 'train_steps_per_second': 5.004, 'total_flos': 2251809994309632.0, 'train_loss': 1.0499560028076171, 'epoch': 1.0})

In [ ]:
print("\n--- FINE-TUNED MODEL OUTPUT (AFTER FINE-TUNING) ---")

# Generate text with the fine-tuned model
outputs = peft_model.generate(
    **inputs,
    max_new_tokens=80,
    pad_token_id=tokenizer.eos_token_id # Set pad token for open-ended generation
)

after_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(after_output)


--- FINE-TUNED MODEL OUTPUT (AFTER FINE-TUNING) ---
Write a Python function to implement bubble sort.
Output: def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        for j in range(n-i-1):
            if arr[j] > arr[j+1] :
                arr[j], arr[j+1] = arr[j+1], arr[j]
    


In [ ]:
# Another example prompt
prompt_2 = "Write a Python function that calculates the factorial of a number."

# Tokenize the new prompt and move to the same device as the model
inputs_2 = tokenizer(prompt_2, return_tensors="pt").to(peft_model.device)

print("\n--- FINE-TUNED MODEL OUTPUT (SECOND EXAMPLE) ---")

# Generate text with the fine-tuned model
outputs_2 = peft_model.generate(
    **inputs_2,
    max_new_tokens=64,
    pad_token_id=tokenizer.eos_token_id # Set pad token for open-ended generation
)

after_output_2 = tokenizer.decode(outputs_2[0], skip_special_tokens=True)
print(after_output_2)


--- FINE-TUNED MODEL OUTPUT (SECOND EXAMPLE) ---
Write a Python function that calculates the factorial of a number.
Output: def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

print(factorial(5))

# Output:
# 5! = 5 * 4 * 3 * 2 * 1


In [ ]:
# Sorting example prompt
prompt_sorting = "Write a Python function to implement bubble sort."

# Tokenize the new prompt and move to the same device as the model
inputs_sorting = tokenizer(prompt_sorting, return_tensors="pt").to(peft_model.device)

print("\n--- FINE-TUNED MODEL OUTPUT (SORTING EXAMPLE) ---")

# Generate text with the fine-tuned model
outputs_sorting = peft_model.generate(
    **inputs_sorting,
    max_new_tokens=128, # Increase max_new_tokens for potentially longer code
    pad_token_id=tokenizer.eos_token_id # Set pad token for open-ended generation
)

sorting_output = tokenizer.decode(outputs_sorting[0], skip_special_tokens=True)
print(sorting_output)


--- FINE-TUNED MODEL OUTPUT (SORTING EXAMPLE) ---
Write a Python function to implement bubble sort.
Output: def bubble_sort(arr):
    for i in range(len(arr)):
        for j in range(len(arr)-1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
    return arr

print(bubble_sort([5,4,3,2,1]))

# Output: [1, 2, 3, 4, 5]
# Output: [1, 2, 3, 4, 5]



In [ ]:
!zip -r /content/codegen-python-finetuned /content/codegen-python-finetuned

  adding: content/codegen-python-finetuned/ (stored 0%)
  adding: content/codegen-python-finetuned/checkpoint-1000/ (stored 0%)
  adding: content/codegen-python-finetuned/checkpoint-1000/added_tokens.json (deflated 84%)
  adding: content/codegen-python-finetuned/checkpoint-1000/scheduler.pt (deflated 62%)
  adding: content/codegen-python-finetuned/checkpoint-1000/training_args.bin (deflated 54%)
  adding: content/codegen-python-finetuned/checkpoint-1000/optimizer.pt (deflated 8%)
  adding: content/codegen-python-finetuned/checkpoint-1000/rng_state.pth (deflated 26%)
  adding: content/codegen-python-finetuned/checkpoint-1000/scaler.pt (deflated 64%)
  adding: content/codegen-python-finetuned/checkpoint-1000/vocab.json (deflated 59%)
  adding: content/codegen-python-finetuned/checkpoint-1000/adapter_config.json (deflated 56%)
  adding: content/codegen-python-finetuned/checkpoint-1000/tokenizer.json (deflated 82%)
  adding: content/codegen-python-finetuned/checkpoint-1000/trainer_state.js

In [ ]:
!pip install huggingface_hub

**Reasoning**:
Authenticate with Hugging Face Hub by importing and calling `notebook_login`.



In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Replace "your_username" with your actual Hugging Face username
repo_name = "Rohannso/codegen-python-finetuned"
peft_model.push_to_hub(repo_name, save_adapter=True)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...finetuned/adapter_model.safetensors:  11%|#         |  553kB / 5.25MB            

CommitInfo(commit_url='https://huggingface.co/Rohannso/codegen-python-finetuned/commit/a18741746175164244658cd01d37550a7380ee8b', commit_message='Upload model', commit_description='', oid='a18741746175164244658cd01d37550a7380ee8b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Rohannso/codegen-python-finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='Rohannso/codegen-python-finetuned'), pr_revision=None, pr_num=None)